In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
ckey = 'o7qidRODRWm5u07PtAWrCpSqf'
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV'
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc'
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

api = tweepy.API(auth)

In [104]:
def TweetTemizle(Dataset):
    print ("Tweetler Temizleniyor...")

    pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
    for i in range(len(Dataset["Tweet_Degistirilmis"])):        
        Dataset["Tweet_Degistirilmis"][i]=pattern.sub(' ', Dataset["Tweet_Degistirilmis"][i])
        
    print ("Tweetler Temizlendi!")
    return Dataset

In [114]:
def Ceviri(DataSet):
    print ("Tweetler Çeviriliyor...")
    
    for yazi in DataSet["Tweet_Degistirilmis"]:
        Ceviri_Blob=TextBlob(yazi)
        Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')
        DataSet["Tweet_Ceviri"][yazi]=Eng_Blob
        print(Eng_Blob)    
    print ("Tweetler Çevirildi!")
    return DataSet

In [115]:
def KelimeAyristirma(DataSet):
    data_test = DataSet.copy()
    stop = stopwords.words('turkish')
    data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].str.lower().str.split()
    data_test["Tweet_Degistirilmis"] = data_test["Tweet_Degistirilmis"].apply(lambda x: [item for item in x if item not in stop])

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity 

In [116]:
def toDataFrame(tweets):
    print ("Veriler Kayıt Ediliyor...")
    # COnvert to data frame    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    DataSet = pd.DataFrame()


    DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
    DataSet['Tweet'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]
    DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
    DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
    DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
    DataSet['Dil'] = [tweet.lang for tweet in tweets]
    
    """"
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets] 
    users_retweeted = []    
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    
    DataSet['TweetPlace'] = [i for i in tweets_place]
    DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]
    """
    print ("Veriler Kayıt Edildi!")
    return DataSet

In [117]:
if (not api):
    print ("Bağlantı Yapılamadı!")
    sys.exit(-1)
else:
    print ("Bağlantı Yapıldı!")
    
    aranan=input("Aranacak Kelimeyi Girin: ")
    
    print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
    
    cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr')
    
    results=[]
    for item in cursor.items(10): # Remove the limit to 1000
            results.append(item)
            
    print ("Kelimeye Uygun Tweetler Sorgulandı!")
    
    DataSet = toDataFrame(results)  
    TweetTemizle(DataSet) 
    Ceviri(DataSet)    
    
    DataSet.to_csv('tw.csv',index=False)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
Tweetler Çeviriliyor...


ValueError: Length of values (48) does not match length of index (10)

In [ ]:
Veri=pd.read_csv('tw.csv')

Veri.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
0,1599179527078305792,@yilmazsozcu Toplantıda konuşulanlardan bir şeyler anlayıp onu eleştirmek için bir parça bilgi ve çalışan kafa lazı… https://t.co/DDCp9NYQ4E,Toplantıda konuşulanlardan bir şeyler anlayıp onu eleştirmek için bir parça bilgi ve çalışan kafa lazı,@yilmazsozcu Toplantıda konuşulanlardan bir şeyler anlayıp onu eleştirmek için bir parça bilgi ve çalışan kafa lazı… https://t.co/DDCp9NYQ4E,2022-12-03 23:11:25+00:00,146869913,Sarmutlu #OneLove 🏳️‍🌈,tr
1,1599178439797317633,@ProfDemirtas Abartı değildir böyle bir uçak üstünlük getiriyor doğrudur çok büyük bir avantajdır ama MUTLAK üstünl… https://t.co/eS7W0OTeeK,Abartı değildir böyle bir uçak üstünlük getiriyor doğrudur çok büyük bir avantajdır ama MUTLAK üstünl,@ProfDemirtas Abartı değildir böyle bir uçak üstünlük getiriyor doğrudur çok büyük bir avantajdır ama MUTLAK üstünl… https://t.co/eS7W0OTeeK,2022-12-03 23:07:06+00:00,1152886988665708544,Djem,tr
2,1599153535173922816,@sivriikalemler Üçgendeki tezgah - A. Cem Ersever,Üçgendeki tezgah A Cem Ersever,@sivriikalemler Üçgendeki tezgah - A. Cem Ersever,2022-12-03 21:28:08+00:00,1414530526300545035,Wings of Tears,tr
3,1599153294877741057,@say_cem 4 tane tricky soru sordum 4’unde de patladı. Daha eğitilmesi lazım. https://t.co/M3jM4QD7mx,tane tricky soru sordum unde de patladı Daha eğitilmesi lazım,@say_cem 4 tane tricky soru sordum 4’unde de patladı. Daha eğitilmesi lazım. https://t.co/M3jM4QD7mx,2022-12-03 21:27:10+00:00,3321361180,Ilker buzluk,tr
4,1599148059883032576,"@atakanyeniceli1 @say_cem Hmm aslında hayır, yön verebilyorsunuz ai'ya",Hmm aslında hayır yön verebilyorsunuz ai ya,"@atakanyeniceli1 @say_cem Hmm aslında hayır, yön verebilyorsunuz ai'ya",2022-12-03 21:06:22+00:00,1705527578,Civan,tr


In [ ]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
5,1599140098284494848,Cem Adrian Summertime bana hiç hissetmediğim duyguları hissettiriyor.. \n#cemadrian,Cem Adrian Summertime bana hiç hissetmediğim duyguları hissettiriyor,Cem Adrian Summertime bana hiç hissetmediğim duyguları hissettiriyor.. \n#cemadrian,2022-12-03 20:34:44+00:00,1350142831751528449,Z,tr
6,1599139619622117377,@MuratAykul Türkçe basılı yayınlar içerisinde hala nasıl olup da varlığını sürdürdüğünü anlamadığım bi garabet bu d… https://t.co/BbdjgYdGnq,Türkçe basılı yayınlar içerisinde hala nasıl olup da varlığını sürdürdüğünü anlamadığım bi garabet bu d,@MuratAykul Türkçe basılı yayınlar içerisinde hala nasıl olup da varlığını sürdürdüğünü anlamadığım bi garabet bu d… https://t.co/BbdjgYdGnq,2022-12-03 20:32:50+00:00,106490304,ZeynepRan🐞,tr
7,1599134720880541696,"@ProfDemirtas Ya bu trol tayfanın soy adlarıda iileriyle uyumlu bi şekilde, necmettin ekonomi batır el, cem küçük 😂🤫",Ya bu trol tayfanın soy adlarıda iileriyle uyumlu bi şekilde necmettin ekonomi batır el cem küçük 😂🤫,"@ProfDemirtas Ya bu trol tayfanın soy adlarıda iileriyle uyumlu bi şekilde, necmettin ekonomi batır el, cem küçük 😂🤫",2022-12-03 20:13:22+00:00,51018871,Sapiensoğlu,tr
8,1599134429066059777,@dincerica Mehmet Öz göçmen de Cem Özdemir göçmen değil mi? Keşke bu konuya da açıklık getirse.,Mehmet Öz göçmen de Cem Özdemir göçmen değil mi Keşke bu konuya da açıklık getirse,@dincerica Mehmet Öz göçmen de Cem Özdemir göçmen değil mi? Keşke bu konuya da açıklık getirse.,2022-12-03 20:12:13+00:00,1433516571935248386,Muhammed,tr
9,1599131482143850496,@torbaciniz31 Almazsan ananı varya o ananı g. ... öpüp. S......Cem,Almazsan ananı varya o ananı g öpüp S Cem,@torbaciniz31 Almazsan ananı varya o ananı g. ... öpüp. S......Cem,2022-12-03 20:00:30+00:00,1457730194266075143,gladyonun kraliçesi ip.siz Maral,tr


In [ ]:
KelimeAyristirma(Veri)


In [ ]:
savedTweets = Veri
savedTweets['Subjectivity'] = savedTweets['Tweet_Ceviri'].apply(getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Ceviri'].apply(getPolarity)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
0,1599179527078305792,@yilmazsozcu Toplantıda konuşulanlardan bir şeyler anlayıp onu eleştirmek için bir parça bilgi ve çalışan kafa lazı… https://t.co/DDCp9NYQ4E,Toplantıda konuşulanlardan bir şeyler anlayıp onu eleştirmek için bir parça bilgi ve çalışan kafa lazı,@yilmazsozcu Toplantıda konuşulanlardan bir şeyler anlayıp onu eleştirmek için bir parça bilgi ve çalışan kafa lazı… https://t.co/DDCp9NYQ4E,2022-12-03 23:11:25+00:00,146869913,Sarmutlu #OneLove 🏳️‍🌈,tr,0.0,0.0
1,1599178439797317633,@ProfDemirtas Abartı değildir böyle bir uçak üstünlük getiriyor doğrudur çok büyük bir avantajdır ama MUTLAK üstünl… https://t.co/eS7W0OTeeK,Abartı değildir böyle bir uçak üstünlük getiriyor doğrudur çok büyük bir avantajdır ama MUTLAK üstünl,@ProfDemirtas Abartı değildir böyle bir uçak üstünlük getiriyor doğrudur çok büyük bir avantajdır ama MUTLAK üstünl… https://t.co/eS7W0OTeeK,2022-12-03 23:07:06+00:00,1152886988665708544,Djem,tr,0.0,0.0
2,1599153535173922816,@sivriikalemler Üçgendeki tezgah - A. Cem Ersever,Üçgendeki tezgah A Cem Ersever,@sivriikalemler Üçgendeki tezgah - A. Cem Ersever,2022-12-03 21:28:08+00:00,1414530526300545035,Wings of Tears,tr,0.0,0.0
3,1599153294877741057,@say_cem 4 tane tricky soru sordum 4’unde de patladı. Daha eğitilmesi lazım. https://t.co/M3jM4QD7mx,tane tricky soru sordum unde de patladı Daha eğitilmesi lazım,@say_cem 4 tane tricky soru sordum 4’unde de patladı. Daha eğitilmesi lazım. https://t.co/M3jM4QD7mx,2022-12-03 21:27:10+00:00,3321361180,Ilker buzluk,tr,0.0,0.0
4,1599148059883032576,"@atakanyeniceli1 @say_cem Hmm aslında hayır, yön verebilyorsunuz ai'ya",Hmm aslında hayır yön verebilyorsunuz ai ya,"@atakanyeniceli1 @say_cem Hmm aslında hayır, yön verebilyorsunuz ai'ya",2022-12-03 21:06:22+00:00,1705527578,Civan,tr,0.0,0.0


In [ ]:
savedTweets.tail()

,Tweet_Id,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
5,1599140098284494848,Cem Adrian Summertime bana hiç hissetmediğim duyguları hissettiriyor.. \n#cemadrian,Cem Adrian Summertime bana hiç hissetmediğim duyguları hissettiriyor,Cem Adrian Summertime bana hiç hissetmediğim duyguları hissettiriyor.. \n#cemadrian,2022-12-03 20:34:44+00:00,1350142831751528449,Z,tr,0.0,0.0
6,1599139619622117377,@MuratAykul Türkçe basılı yayınlar içerisinde hala nasıl olup da varlığını sürdürdüğünü anlamadığım bi garabet bu d… https://t.co/BbdjgYdGnq,Türkçe basılı yayınlar içerisinde hala nasıl olup da varlığını sürdürdüğünü anlamadığım bi garabet bu d,@MuratAykul Türkçe basılı yayınlar içerisinde hala nasıl olup da varlığını sürdürdüğünü anlamadığım bi garabet bu d… https://t.co/BbdjgYdGnq,2022-12-03 20:32:50+00:00,106490304,ZeynepRan🐞,tr,0.0,0.0
7,1599134720880541696,"@ProfDemirtas Ya bu trol tayfanın soy adlarıda iileriyle uyumlu bi şekilde, necmettin ekonomi batır el, cem küçük 😂🤫",Ya bu trol tayfanın soy adlarıda iileriyle uyumlu bi şekilde necmettin ekonomi batır el cem küçük 😂🤫,"@ProfDemirtas Ya bu trol tayfanın soy adlarıda iileriyle uyumlu bi şekilde, necmettin ekonomi batır el, cem küçük 😂🤫",2022-12-03 20:13:22+00:00,51018871,Sapiensoğlu,tr,0.0,0.0
8,1599134429066059777,@dincerica Mehmet Öz göçmen de Cem Özdemir göçmen değil mi? Keşke bu konuya da açıklık getirse.,Mehmet Öz göçmen de Cem Özdemir göçmen değil mi Keşke bu konuya da açıklık getirse,@dincerica Mehmet Öz göçmen de Cem Özdemir göçmen değil mi? Keşke bu konuya da açıklık getirse.,2022-12-03 20:12:13+00:00,1433516571935248386,Muhammed,tr,0.0,0.0
9,1599131482143850496,@torbaciniz31 Almazsan ananı varya o ananı g. ... öpüp. S......Cem,Almazsan ananı varya o ananı g öpüp S Cem,@torbaciniz31 Almazsan ananı varya o ananı g. ... öpüp. S......Cem,2022-12-03 20:00:30+00:00,1457730194266075143,gladyonun kraliçesi ip.siz Maral,tr,0.0,0.0
